In [ ]:
import numpy as np
import copy
import networkx as nx
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Funciones Influencia Centro-Periferia

In [ ]:
def pandas_2_graph(df):
    G = nx.DiGraph()
    for row in df.iterrows():
        G.add_weighted_edges_from([(row[1]['Importer'], row[1]['Exporter'], row[1]['Value'])])
    return G

In [ ]:
def normalize_DiGraph(G):
  mapa_labels = dict(zip([i for i in range(len(list(G.nodes())))], list(G.nodes())))
  matrix = nx.to_numpy_array(G)
  normalized_matrix = (matrix.T/matrix.sum(axis=1)).T
  normalized_graph = nx.from_numpy_array(normalized_matrix, create_using=nx.DiGraph())
  normalized_graph = nx.relabel_nodes(normalized_graph, mapa_labels)
  return normalized_graph

In [ ]:
def elasticidades(G):

  matriz = prematriz = nx.to_numpy_array(G)
  matriz_propension = np.diag(list(nx.get_node_attributes(G, 'propension').values()))
  matriz_ingreso = np.diag(list(nx.get_node_attributes(G, 'ingreso').values()))

  E = matriz_ingreso @ matriz_propension @ matriz @ np.linalg.inv(np.identity(len(matriz)) - matriz_propension @ matriz) @ np.linalg.inv(matriz_ingreso)

  mapa_labels = dict(zip([i for i in range(len(list(G.nodes())))], list(G.nodes())))
  TI = nx.from_numpy_array(E, create_using=nx.DiGraph())
  TI = nx.relabel_nodes(TI, mapa_labels)

  nx.set_node_attributes(TI, dict(G.nodes(data='propension')), "propension")
  nx.set_node_attributes(TI, dict(G.nodes(data='propension')), "ingreso")

  return E, TI

In [ ]:
def participacion(ingreso, grupo):

  ingresos_grupo = 0.0
  participaciones = []

  for nodo in ingreso:
    if nodo in grupo:
      valor = ingreso[nodo]
      ingresos_grupo += valor
      participaciones.append(valor)
    else:
      participaciones.append(0.0)

  return dict(zip(ingreso.keys(), np.array(participaciones)/ingresos_grupo))

In [ ]:
def TI_ng(E, ingreso, grupo):
  wg = participacion(ingreso, grupo)
  return dict(zip(wg.keys(), E @ np.array(list(wg.values()))))

def TI_gn(E, ingreso, grupo):
  wg = participacion(ingreso, grupo)
  return dict(zip(wg.keys(), E.T @ np.array(list(wg.values()))))

In [ ]:
def TI_gg(E, ingreso, grupo, grupo_prima):
  wg = participacion(ingreso, grupo)
  wg_prima = participacion(ingreso, grupo_prima)

  return np.array(list(wg.values())).T @ E @ np.array(list(wg_prima.values()))

def RTF_g(E, ingreso, propension, grupo):
  wg = participacion(ingreso, grupo)

  len_grupo = np.count_nonzero(np.array(list(wg.values())))

  TF_g = TI_gg(E, ingreso, grupo, grupo)

  TF_g_H = len_grupo * (np.array(list(wg.values())).T @ np.diag(list(propension.values())) @  np.array(list(wg.values()))) / (len(E) * (1- np.mean(list(propension.values()))))

  return TF_g/TF_g_H

In [ ]:
def BTP_gg(E, ingreso, propension, grupo, grupo_prima):

  return RTF_g(E, ingreso, propension, grupo) * TI_gg(E, ingreso, grupo, grupo_prima) - RTF_g(E, ingreso, propension, grupo_prima) * TI_gg(E, ingreso, grupo_prima, grupo)

In [ ]:
def estructuras_centro_perifericas(TI, paises): # se pone paises para luego calcular estructuras centro-perifericas de subconjuntos de paises
  
  TIc = copy.deepcopy(TI)

  E = nx.to_numpy_array(TIc)
  propension = nx.get_node_attributes(TIc, 'propension')
  ingreso = nx.get_node_attributes(TIc, 'ingreso')

  influencias = dict.fromkeys(paises)

  for pais in paises:

    paises_restantes = [x for x in paises if x != pais]
    influencias[pais] = TI_gg(E, ingreso, [pais], paises_restantes)

  influencias = dict(sorted(influencias.items(), key = lambda item: item[1], reverse = True))

  particiones = {}
  q = 0

  while len(influencias) > 2:

    particiones[q] = {'BTP': 0.0, 'nodos': None}

    centro = []
    
    for pais in list(influencias.keys())[:-1]:

      centro.append(pais)

      periferia = [x for x in paises if x not in centro]

      BTP = BTP_gg(E, ingreso, propension, centro, periferia)

      if particiones[q]['BTP'] < BTP: # ¿será que los BTP de diferentes grupos pueden ser iguales?

        particiones[q]['BTP'] = BTP
        particiones[q]['nodos'] = centro.copy() 
    
    for nodo in particiones[q]['nodos']: influencias.pop(nodo)
    TIc.remove_nodes_from(particiones[q]['nodos'])
    
    E = nx.to_numpy_array(TIc)
    propension = nx.get_node_attributes(TIc, 'propension')
    ingreso = nx.get_node_attributes(TIc, 'ingreso')

    q = q + 1

  return particiones

In [ ]:
def centro_periferia_promedio(estructuras):

  niveles = {}

  for nivel in estructuras:
    niveles[nivel] = estructuras[nivel]['BTP']

  promedio_BTP = np.mean(list(niveles.values()))

  centro = []
  periferia = []

  for nivel in niveles:
    if niveles[nivel] >= promedio_BTP:
      centro += estructuras[nivel]['nodos']
    elif niveles[nivel] < promedio_BTP:
      periferia += estructuras[nivel]['nodos']
    else:
      print('Error al Identificar Centro y Periferia')
  
  return centro, periferia

#Importar-Preparar-Guardar Grafo Influencias

In [ ]:
df_pre_pib = pd.read_csv('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Datos/World_Bank_Data/NY.GDP.MKTP.CD_1995-2022.csv')
df_pre_propensiones = pd.read_csv('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Datos/World_Bank_Data/NE.IMP.GNFS.ZS_1995-2022.csv')

for año in range(año_inicial, año_final + 1):

  print(año)
  #df_pre_enriched = pd.read_csv('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Datos/Processed_Data/BACI_HS92_V202301/Y'+ str(año) +'/Enriched_BACI_HS92_Y'+ str(año) +'_V202301.csv')
  df_pre_merged = pd.read_csv('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Datos/Processed_Data/BACI_HS92_V202301/Y'+ str(año) +'/Merged_BACI_HS92_Y'+ str(año) +'_V202301.csv')

  pre_grafo = pandas_2_graph(df_pre_merged)

  df_propensiones = df_pre_propensiones.loc[df_pre_propensiones["date"] == año, ['countryiso3code', 'value']]
  df_propensiones['value'] = df_propensiones['value'].div(100.)
  table_propensiones = pd.pivot_table(df_propensiones, values = 'value', index = ['countryiso3code'])
  propensiones_dict = table_propensiones.set_index(table_propensiones.index).T.to_dict('index')['value']

  df_pib = df_pre_pib.loc[df_pre_pib["date"] == año, ['countryiso3code', 'value']]
  df_pib['value'] = df_pib['value'].div(1000.)
  table_pib = pd.pivot_table(df_pib, values = 'value', index = ['countryiso3code'])
  pib_dict = table_pib.set_index(table_pib.index).T.to_dict('index')['value']

  propension = {}
  ingreso_pib = {}
  excepciones = [] # paises de la red que no registran un pib y propension a importar en los datos que tenemos del banco mundial

  for country in pre_grafo.nodes():
    if (country in pib_dict) and (country in propensiones_dict):
      propension[country] = propensiones_dict[country]
      ingreso_pib[country] = pib_dict[country]
    else:
      excepciones.append(country)

  pre_grafo.remove_nodes_from(excepciones)
  grafo = normalize_DiGraph(pre_grafo)
  nx.set_node_attributes(grafo, propension, "propension")
  nx.set_node_attributes(grafo, ingreso_pib, "ingreso")

  E, TI = elasticidades(grafo) # Meter el grafo cargado

  nx.write_graphml(TI, '/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/TI2'+ str(año) +'.graphml')


# Generar y Guardar Datos de Centro-Periferia

In [ ]:
año_inicial = 1999
año_final = 2021

In [ ]:
for año in range(año_inicial, año_final + 1):

  print(año)
  TI = nx.read_graphml('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/TI2'+ str(año) +'.graphml')

  estructuras = estructuras_centro_perifericas(TI, list(TI.nodes()))
  centro, periferia = centro_periferia_promedio(estructuras)

  with open('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/CentroPeriferia_promedio2_'+ str(año) +'.json', "w") as f:
    json.dump({'centro': centro, 'periferia': periferia}, f)

  with open('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/estructuras2_'+ str(año) +'.json', "w") as f:
    json.dump(estructuras, f)

In [ ]:
grafos_TI = {}
centro_periferia_TI = {}
estructuras_TI = {}

df_resultados_influencia_promedio = pd.DataFrame()

for año in range(año_inicial, año_final + 1):

  print(año)
  TI = nx.read_graphml('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/TI2'+ str(año) +'.graphml')
  E = nx.to_numpy_array(TI)

  with open('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/CentroPeriferia_promedio2_'+ str(año) +'.json', "r") as f:
    centro_periferia_promedios = json.load(f)  

  mundo = list(TI.nodes())

  centro_promedio = centro_periferia_promedios['centro']
  periferia_promedio = [x for x in mundo if x not in centro_promedio]#centro_periferia_promedios['periferia']
  

  ingresos = nx.get_node_attributes(TI, "ingreso")
  propensiones = nx.get_node_attributes(TI, "propension")

  dic_medidas_promedio = {'BTP_c2p' : [BTP_gg(E, ingresos, propensiones, centro_promedio, periferia_promedio)], 'TI_c2p' : [TI_gg(E, ingresos, centro_promedio, periferia_promedio)], 
                 'TI_p2c' : [TI_gg(E, ingresos, periferia_promedio, centro_promedio)], 'TI_c2c' : [TI_gg(E, ingresos, centro_promedio, centro_promedio)], 
                 'RTF_c2c' : [RTF_g(E, ingresos, propensiones, centro_promedio)], 'TI_p2p' : [TI_gg(E, ingresos, periferia_promedio, periferia_promedio)], 
                 'RTF_p2p' : [RTF_g(E, ingresos, propensiones, periferia_promedio)], 'TI_c2m' : [TI_gg(E, ingresos, centro_promedio, mundo)], 
                 'TI_p2m' : [TI_gg(E, ingresos, periferia_promedio, mundo)], 'TI_m2m' : [TI_gg(E, ingresos, mundo, mundo)],
                 'TI_m2p' : [TI_gg(E, ingresos, mundo, periferia_promedio)], 'TI_m2c' : [TI_gg(E, ingresos, mundo, centro_promedio)], 
                 'RTF_m2m' : [RTF_g(E, ingresos, propensiones, mundo)]}

  for categoria in dic_medidas_promedio:
    df_fila_promedio = pd.DataFrame({'año' : [año], 'medida' : dic_medidas_promedio[categoria], 'categoria' : [categoria]})
    df_resultados_influencia_promedio = pd.concat([df_resultados_influencia_promedio, df_fila_promedio])

In [ ]:
df_resultados_influencia_promedio.to_csv('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/' + str(año_inicial) + '_to_' + str(año_final) + 'influencias_promedio.csv', index=False)

# Analisis de Centro-Periferia en el Tiempo

In [ ]:
año_inicial = 1999
año_final = 2021

# 'BTP_c2p', 'TI_c2p', 'TI_p2c', 'TI_c2c', 'RTF_c2c', 'TI_p2p', 'RTF_p2p', 'TI_c2m', 'TI_p2m', 'TI_m2m', 'TI_m2p', 'TI_m2c', 'RTF_m2m'

In [ ]:
df_influencias_promedio = pd.read_csv('/content/drive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/' + str(año_inicial) + '_to_' + str(año_final) + 'influencias_promedio.csv')

In [ ]:
filter = ['BTP_c2p', 'RTF_c2c', 'RTF_p2p', 'TI_c2p', 'TI_p2c', 'TI_c2c', 'TI_p2p', 'TI_c2m', 'TI_p2m', 'TI_m2m', 'TI_m2p', 'TI_m2c', 'RTF_m2m']

df_filtered_promedio = df_influencias_promedio[df_influencias_promedio["categoria"].isin(filter)]

In [ ]:
fig = px.line(df_filtered_promedio, x="año", y="medida", color='categoria', title="Promedio")
fig.show()